In [ ]:
import os
import json
from google.colab import auth, userdata, drive
from google.cloud import storage
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from openai import OpenAI
import io

# Google認証を行う（スコープを指定）
auth.authenticate_user()

# 環境変数の設定
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

print("認証完了。環境変数設定完了。")

In [ ]:
# Colabのシークレット機能を使用してフォルダIDを取得
FOLDER_ID = userdata.get('FOLDER_ID')
GCS_BUCKET_NAME = 'chatbot-input-documents'

# クライアントの初期化
drive_service = build('drive', 'v3', cache_discovery=False)
openai_client = OpenAI()  # API keyは環境変数から自動的に読み込まれます
storage_client = storage.Client()

print(f"FOLDER_ID: {FOLDER_ID}")
print(f"GCS_BUCKET_NAME: {GCS_BUCKET_NAME}")
print("クライアント初期化完了。")

In [ ]:
# クライアントの初期化
drive_service = build('drive', 'v3', cache_discovery=False)
openai_client = OpenAI()  # API keyは環境変数から自動的に読み込まれます
storage_client = storage.Client()

In [ ]:
def download_document(file_id):
    request = drive_service.files().export_media(fileId=file_id, mimeType='text/plain')
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    return fh.getvalue().decode('utf-8')

def list_documents_in_folder(folder_id):
    query = f"'{folder_id}' in parents and mimeType='application/vnd.google-apps.document'"
    results = drive_service.files().list(q=query, spaces='drive', fields='files(id, name)').execute()
    return results.get('files', [])

def vectorize_text(text):
    response = openai_client.embeddings.create(input=text, model="text-embedding-3-small")
    return response.data[0].embedding

def save_to_gcs(data, filename):
    bucket = storage_client.bucket(GCS_BUCKET_NAME)
    blob = bucket.blob(filename)
    blob.upload_from_string(json.dumps(data))
    print(f"{filename} をGCSバケット {GCS_BUCKET_NAME} に保存しました")

print("ユーティリティ関数定義完了。")

In [ ]:
documents = list_documents_in_folder(FOLDER_ID)
print(f"フォルダ内のドキュメント数: {len(documents)}")
for doc in documents:
    print(f"ドキュメントID: {doc['id']}, 名前: {doc['name']}")

In [ ]:
document_vectors = {}

for document in documents:
    doc_id, doc_name = document['id'], document['name']
    print(f"ドキュメント処理中: {doc_name}")
    
    doc_content = download_document(doc_id)
    print(f"内容の長さ: {len(doc_content)} 文字")
    
    vector = vectorize_text(doc_content)
    document_vectors[doc_name] = vector
    print(f"ベクトルの長さ: {len(vector)}")
    print("---")

print(f"処理したドキュメントの総数: {len(document_vectors)}")

In [ ]:
# ドキュメントベクトルをGCSに保存
save_to_gcs(document_vectors, 'document_vectors.json')

# 例：各ベクトルの最初の10要素を表示
for doc_name, vector in document_vectors.items():
    print(f"ドキュメント: {doc_name}, ベクトル: {vector[:10]}...")